In [1]:
#pip install findspark
#/usr/local/opt/apache-spark/libexec/sbin
#./start-thriftserver.sh

In [1]:
import findspark
findspark.init()

In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder\
    .appName("spark").config('spark.driver.memory', '32g')\
    .config("hive.server2.thrift.port", 10000)\
    .config("spark.sql.hive.thriftServer.singleSession", True)\
    .enableHiveSupport()\
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/05 22:48:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType,TimestampType
from pyspark.sql.types import DateType
from pyspark.sql.functions import *
from pyspark.files import SparkFiles

schema = StructType([
    StructField("ID", IntegerType()),
    StructField("Speed", DoubleType()),
    StructField("TravelTime", IntegerType()),
    StructField("Status", IntegerType()),
    StructField("timedate", StringType()),
    StructField("LinkId", IntegerType()),
    StructField("LinkPoints", StringType()),
    StructField("EncodedLinkPoints", StringType()),
    StructField("EncodedPolyLineLvls", StringType()),
    StructField("Owner", StringType()),
    StructField("TranscomId", IntegerType()),
    StructField("Borough", StringType()),
    StructField("Link_Name", StringType())
])

#spark.sparkContext.addFile("/Users/minoseah629/Repos/AIT614Project/Data_Preparation/Python/Raw_Data/raw.csv")
#file://Users/minoseah629/Repos/AIT614Project/Data_Preparation/Python/Raw_Data/raw.csv
schema1 = "ID bigint, Speed float, TravelTime int, Status int, TimeDate string, LinkId int, LinkPoints string, EncodedLinkPoints string, EncodedPolyLineLvls string, Owner string, TranscomId int, Borough string,Link_Name string"
df2 = spark.read.format('csv').option("header", "true").schema(schema).load('/Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv')
#.csv(SparkFiles.get("raw.csv"), schema=schema,header=True)
df2.printSchema()
# .option("inferSchema", "true")

root
 |-- ID: integer (nullable = true)
 |-- Speed: double (nullable = true)
 |-- TravelTime: integer (nullable = true)
 |-- Status: integer (nullable = true)
 |-- timedate: string (nullable = true)
 |-- LinkId: integer (nullable = true)
 |-- LinkPoints: string (nullable = true)
 |-- EncodedLinkPoints: string (nullable = true)
 |-- EncodedPolyLineLvls: string (nullable = true)
 |-- Owner: string (nullable = true)
 |-- TranscomId: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Link_Name: string (nullable = true)



In [3]:
df2 = df2.withColumn("timedate", to_timestamp("timedate", 'MM/dd/yyyy hh:mm:ss a'))
df2.na.drop()
df2.createOrReplaceTempView("dataset")

df2.count()
df2.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Speed: double (nullable = true)
 |-- TravelTime: integer (nullable = true)
 |-- Status: integer (nullable = true)
 |-- timedate: timestamp (nullable = true)
 |-- LinkId: integer (nullable = true)
 |-- LinkPoints: string (nullable = true)
 |-- EncodedLinkPoints: string (nullable = true)
 |-- EncodedPolyLineLvls: string (nullable = true)
 |-- Owner: string (nullable = true)
 |-- TranscomId: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Link_Name: string (nullable = true)



In [4]:
query = "select ID,split(LinkPoints,' ') as `link points` from dataset"

In [5]:
df2=spark.sql(query)
link = df2
link.createOrReplaceTempView('links')

In [6]:
points = spark.sql("select * from links group by id, `link points` order by id asc")
points.printSchema()
points.show()
points.count()

root
 |-- ID: integer (nullable = true)
 |-- link points: array (nullable = true)
 |    |-- element: string (containsNull = true)



22/04/05 22:49:24 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ID, LINK_POINTS
 Schema: ID, LinkPoints
Expected: LinkPoints but found: LINK_POINTS
CSV file: file:///Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv
22/04/05 22:50:47 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ID, LINK_POINTS
 Schema: ID, LinkPoints
Expected: LinkPoints but found: LINK_POINTS
CSV file: file:///Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv


+---+--------------------+
| ID|         link points|
+---+--------------------+
|  1|[40.74047,-74.009...|
|  2|[40.73933,-74.010...|
|  3|[40.76375,-73.999...|
|  4|[40.7607,-74.0021...|
|106|[40.77158,-73.994...|
|107|[40.64289,-74.213...|
|108|[40.62464,-74.177...|
|110|[40.5256,-74.2303...|
|119|[40.70631,-74.015...|
|122|[40.7073904,-74.0...|
|123|[40.70738,-74.015...|
|124|[40.68036,-74.004...|
|126|[40.8271606,-73.8...|
|126|[40.8271606,-73.8...|
|129|[40.8240706,-73.8...|
|137|[40.8242005,-73.8...|
|140|[40.79789,-73.919...|
|141|[40.772251,-73.91...|
|142|[40.83037,-73.850...|
|145|[40.7081105,-73.9...|
+---+--------------------+
only showing top 20 rows



154

In [16]:
import json
d = {}

In [19]:
def test (x: pyspark.Row): 
    dic = []
    i = 1
    for point in x["link points"]:
        y = point.split(",")
        if (len(y)==2):
            if (len(y[0])>2) and (len(y[1])>2):
                dic.append((float(y[1]), float(y[0])))
                i=i+1
    d[x['Id']]={"start": {"type": "Point", "coordinates":dic[0]}, "end": {"type": "Point", "coordinates":dic[-1]}, "points": {"type": "MultiPoint","coordinates":dic[1:-1] }}

links = points.select('Id','link points').collect()#.foreach(lambda x : test(x))

for x in links: 
    test(x)

22/04/06 00:00:26 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ID, LINK_POINTS
 Schema: ID, LinkPoints
Expected: LinkPoints but found: LINK_POINTS
CSV file: file:///Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv


In [20]:
display(d)

{1: {'start': {'type': 'Point', 'coordinates': (-74.009251, 40.74047)},
  'end': {'type': 'Point', 'coordinates': (-74.002631, 40.76003)},
  'points': {'type': 'MultiPoint',
   'coordinates': [(-74.00893, 40.74137),
    (-74.008591, 40.7431706),
    (-74.00797, 40.7462304),
    (-74.007651, 40.74812),
    (-74.007691, 40.748701),
    (-74.00819, 40.74971),
    (-74.008321, 40.75048),
    (-74.00789, 40.751611),
    (-74.00704, 40.7537504),
    (-74.00463, 40.75721)]}},
 2: {'start': {'type': 'Point', 'coordinates': (-74.01004, 40.73933)},
  'end': {'type': 'Point', 'coordinates': (-74.011131, 40.72619)},
  'points': {'type': 'MultiPoint',
   'coordinates': [(-74.01012, 40.73895),
    (-74.010021, 40.7376),
    (-74.01026, 40.7346),
    (-74.010781, 40.72912)]}},
 3: {'start': {'type': 'Point', 'coordinates': (-73.999191, 40.76375)},
  'end': {'type': 'Point', 'coordinates': (-74.0, 40.74114)},
  'points': {'type': 'MultiPoint',
   'coordinates': [(-73.99935, 40.763521),
    (-74.00136,

In [21]:
out_file = open("points.json", "w")
json.dump(d,out_file)

In [35]:
points.write.format('json').mode('overwrite').save('file:/Users/minoseah629/Repos/AIT614Project/points')

22/04/04 22:44:13 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: ID, LINK_POINTS
 Schema: ID, LinkPoints
Expected: LinkPoints but found: LINK_POINTS
CSV file: file:///Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv


In [36]:
spark.stop()